# web框架概述
- web框架和web服务器的关系介绍
    - 前面已经学习过web服务器, 我们知道web服务器主要是接收用户的http请求,根据用户的请求返回不同的资源数据，但是之前我们开发的是静态web服务器，返回的都是静态资源数据，假如我们想要web服务器返回动态资源那么该如何进行处理呢？
    - 这里我们给大家介绍一个web框架，使用web框架专门负责处理用户的动态资源请求，这个web框架其实就是一个为web服务器提供服务的应用程序，简称web框架。
    - 关系说明:
        - web服务器接收浏览器发起的请求，如果是动态资源请求找web框架来处理
        - web框架负责处理浏览器的动态资源请求，把处理的结果发生给web服务器
        - web服务器再把响应结果发生给浏览器
- 静态资源
    - 不需要经常变化的资源，这种资源web服务器可以提前准备好，比如: png/jpg/css/js等文件
- 动态资源
    - 和静态资源相反, 这种资源会经常变化，比如: 我们在京东浏览商品时经常会根据条件进行筛选，选择不同条件, 浏览的商品就不同，这种资源web服务器无法提前准备好，需要web框架来帮web服务器进行准备，在这里web服务器可以把.html的资源请求认为是动态资源请求交由web框架进行处理
- WSGI协议
    - 它是web服务器和web框架之间进行协同工作的一个规则，WSGI协议规定web服务器把动态资源的请求信息传给web框架处理，web框架把处理好的结果返回给web服务器

In [1]:
# 构架模板
# web
import socket
import os
import threading
import sys
import framwork


# http协议的Web服务器类
class HttpWebServer(object):
    def __init__(self, port):
    # 创建tcp服务端套接字
    tcp_server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 设置端口号复用，程序退出端口号立即释放
    tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)
    # 绑定端口号
    tcp_server_socket.bind("", port)
    # 设置监听
    tcp_server_socket.listen(128)
    # 把tcp服务器的套接字作为Web服务器对象的属性
    self.tcp_server_socket = tcp_server_socket
    
    # 处理客户端请求
    @staticmethod
    def handle_client_request(new_socket):
        # 接收客户端的请求信息
        recv_data = new_socket.recv(4096)
        # 判断接收的数据长度是否为0
        if len(recv_data) == 0:
            new_socket.close()
            return

        # 对二进制数据进行解码
        recv_centent = recv_data.decode("utf-8")
        print(recv_centent)

        # 对数据按照空格进行分割
        request_list = recv_centent.split(" ", maxsplit=2)
        # 获取请求的资源路径
        request_path = request_list[1]
        print(request_path)

        # 判断请求的是否是根目录，如果是根目录设置返回的信息
        if request_path == "/":
            request_path = "/index.html"
            
        # 判断是否是动态资源请求，以后后缀是.html的请求任务认为是动态资源请求
        if resquest_path.endswith(".html"):
            """动态资源请求"""
            # 动态资源请求找web框架进行处理，需要把请求参数给web框架
            # 准备给web框架的参数信息，都要放到字典里面
            env = {
                "request_path":resquest_path,
                # 传入请求头信息，额外的参数可以在字典里面再进行添加
            }
            # 使用框架处理动态资源请求
            # 1. web框架需要把处理结果返回给web服务器
            # 2. web服务器负责把返回的结果封装成响应报文发送给浏览器
            status, header, response_body = framwork.handle_request(env)
            print(status, header, respongse_body)
            # 响应行
            response_line = f"HTTP1.1 {status}\r\n"
            # 响应头
            response_header = ""
            for header in headers:
                response_header += f"{header}\r\n"
                
            # 响应报文
            response_data = (response_line +
                             response_header +
                             "\r\n" +
                             response_body).encode("utf8")
            
            # 发送响应报文数据给浏览器
            new_socket.send(response_data)
            # 关闭连接
            new_socket.close()
            
        else:
            # 判断输入的页面是否存在
            # 1. os.path.exits 判断某个文件是否存在
            # os.path.exists("static/" + request_path)
            # 2. try-except

            try:
                # 打开文件读取文件中的数据，提示：这里使用rb模式，兼容打开图片文件
                with open("static" + request_path, "rb") as file: # 这里的file表示打开文件的对象
                    file_data = file.read()
                # 提示：with open 关闭文件这步操作不用程序员来完成，系统帮我们来完成
            except Exception as e:
                # 代码执行到此，说明没有请求的该文件，返回404状态
                # 响应行
                response_line = "HTTP/1.1 200 ok\r\n"
                # 响应头
                response_header = "Server: PWS/1.0\r\n"
                # 读取404页面数据
                with open("static/error.html", "rb") as file:
                    file_data = file.read()
                # 响应体
                response_body = file_data

                # 把数据封装成http响应报文格式的数据
                response = (response_line + 
                            response_header + 
                            "\r\n").encode("utf-8") + response_body
                # 发送给浏览器的相应报文数据
                new_socket.send(response)
            else:    
                # 代码执行到此，说明文件存在，返回200状态信息
                # 响应行
                response_line = "HTTP/1.1 200 ok\r\n"
                # 响应头
                response_header = "Server: PWS/1.0\r\n"
                # 响应体
                response_body = file_data

                # 把数据封装成http响应报文格式的数据
                response = (response_line + 
                            response_header + 
                            "\r\n").encode("utf-8") + response_body

                # 发送给浏览器的响应报文数据
                new_socket.send(response)
            finally:
                # 关闭服务于客户端的套接字
                new_socket.close()
    
    # 启动服务器方法
    def start(self):
         # 循环等待接收客户端的连接请求
        while True:
            # 等待接收客户端的连接请求
            new_socket, ip_port = self.tcp_server_socket.accept()
            # 代码执行到此，说明连接建立成功
            sub_thread = threading.Thread(target=self.handle_client_request, args=(new_socket,))
            # 设置成为守护主线程
            sub_thread.setDaemon(True)
            # 启动子线程执行对应的任务
            sub_thread.start()


def main():
    # 获取终端命令行参数
    # params = sys.argv
    # if len(params) != 2:
        # print("执行的命令格式如下：python3 xxx.py 9000")
        # return
    
    # 判断第二个参数是否都是由数字组成的字符串
    # if not params[1].isdigit():
        # print("执行的命令格式如下：python3 xxx.py 9000")
        # return
    
    # 代码执行到期，说明命令行参数的个数一定是2个，并且第二个参数是由数字组成的字符串
    # port = int(params[1])
    
    # 创建Web服务器
    web_server = HttpWebServer(8000)
    # 启动服务器
    web_server.start()
   
        
# 判断是否是主模块的代码
if __name__ == "__main__":
    main()

IndentationError: expected an indented block (<ipython-input-1-3fc10db53950>, line 13)

In [ ]:
# 模板替换功能开发
# framework
# web框架的职责专门负责处理动态资源请求
import time


# 获取首页数据
def index():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/index.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 处理没有找到的动态资源
def not_found():
    # 状态信息
    status = "404 Not Found"
    # 响应头信息
    response_header = [("Server", "WPS/1.1")]
    # web框架处理后的数据
    data = "Not Found"
    
    # 这里返回的是元组
    return status, response_header, data


# 处理动态资源请求
def handle_request(env):
    # 获取动态的请求资源路径
    request_path = env["request_path"]
    print("动态资源请求的地址"， resquest_path)
    # 判断请求的动态资源路径，选择指定的函数处理对应的动态资源请求
    if request_path == "/index.html":
        # 获取首页数据
        result = index()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result
    else:
        # 没有动态资源数据，返回404状态信息
        result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result

In [ ]:
# 路由列表功能开发
# framework
# web框架的职责专门负责处理动态资源请求
import time


# 获取首页数据
def index():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/index.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 获取个人中心数据
def center():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/center.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 处理没有找到的动态资源
def not_found():
    # 状态信息
    status = "404 Not Found"
    # 响应头信息
    response_header = [("Server", "WPS/1.1")]
    # web框架处理后的数据
    data = "Not Found"
    
    # 这里返回的是元组
    return status, response_header, data


# 路由列表，列表里面的每一条记录都是一个路由
route_list = [
    ("/index.html", index),
    ("center.html", center)
]


# 处理动态资源请求
def handle_request(env):
    # 获取动态的请求资源路径
    request_path = env["request_path"]
    print("动态资源请求的地址"， resquest_path)
    
    # 遍历路由列表，匹配请求的url
    for path, func in route_list:
        if request_path == path:
            # 找到了指定的路由，执行对应的处理函数
            result = func()
            return result
    else:
        # 没有动态资源数据，返回404状态信息
        result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result
    
    # 判断请求的动态资源路径，选择指定的函数处理对应的动态资源请求
    # if request_path == "/index.html":
        # 获取首页数据
        # result = index()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result
    # else:
        # 没有动态资源数据，返回404状态信息
        # result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result

In [ ]:
# 装饰器方式的添加路由
# framework
# web框架的职责专门负责处理动态资源请求
import time


# 路由列表，列表里面的每一条记录都是一个路由
route_list = [
#     ("/index.html", index),
#     ("center.html", center)
]


# 定义带有参数的装饰器
def route(path):
    # 装饰器
    def docorator(func):
        # 当执行装饰器的时候就需要把路由添加到路由列表里面
        # 当装饰函数的时候只添加一次路由即可
        route_list.append((path, func))
        
        def inner():
            result = func()
            return result
        
    # 返回一个装饰器
    return decorator


# 获取首页数据
@route("/index.html")
def index():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/index.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 获取个人中心数据
@route("/center.html")
def center():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/center.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 处理没有找到的动态资源
def not_found():
    # 状态信息
    status = "404 Not Found"
    # 响应头信息
    response_header = [("Server", "WPS/1.1")]
    # web框架处理后的数据
    data = "Not Found"
    
    # 这里返回的是元组
    return status, response_header, data


# 处理动态资源请求
def handle_request(env):
    # 获取动态的请求资源路径
    request_path = env["request_path"]
    print("动态资源请求的地址"， resquest_path)
    
    # 遍历路由列表，匹配请求的url
    for path, func in route_list:
        if request_path == path:
            # 找到了指定的路由，执行对于的处理函数
            result = func()
            return result
    else:
        # 没有动态资源数据，返回404状态信息
        result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result
    
    # 判断请求的动态资源路径，选择指定的函数处理对应的动态资源请求
    # if request_path == "/index.html":
        # 获取首页数据
        # result = index()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result
    # else:
        # 没有动态资源数据，返回404状态信息
        # result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result

In [ ]:
# 显示股票信息页面的开发
# framework
# web框架的职责专门负责处理动态资源请求
import time
import pymysql


# 路由列表，列表里面的每一条记录都是一个路由
route_list = [
#     ("/index.html", index),
#     ("center.html", center)
]


# 定义带有参数的装饰器
def route(path):
    # 装饰器
    def docorator(func):
        # 当执行装饰器的时候就需要把路由添加到路由列表里面
        # 当装饰函数的时候只添加一次路由即可
        route_list.append((path, func))
        
        def inner():
            result = func()
            return result
        
    # 返回一个装饰器
    return decorator


# 获取首页数据
@route("/index.html")
def index():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/index.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    # 创建连接对象
    conn = pymysql.connect(host="localhost", 
                           port="3306", 
                           user="root", 
                           password="mysql", 
                           database="stock_db", 
                           charset="utf8")
    # 获取游标
    cursor = conn.cursor()
    # 准备sql
    sql = "select * from info;"
    # 执行sql
    cursor.execute(sql)
    # 获取查询结果
    result = cursor.fetchall()
    print(result)
    # 关闭游标
    cursor.close()
    # 关闭连接
    conn.close()
    
    # 遍历每一条数据，完成数据的tr标签的封装
    data = ""
    for row in result:
        data += """<tr>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td><input type="button" value="添加" id="toAdd" name="toAdd" systemidvaule="000007"></td>
                </tr>""" % row
        
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 获取个人中心数据
@route("/center.html")
def center():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/center.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 处理没有找到的动态资源
def not_found():
    # 状态信息
    status = "404 Not Found"
    # 响应头信息
    response_header = [("Server", "WPS/1.1")]
    # web框架处理后的数据
    data = "Not Found"
    
    # 这里返回的是元组
    return status, response_header, data


# 处理动态资源请求
def handle_request(env):
    # 获取动态的请求资源路径
    request_path = env["request_path"]
    print("动态资源请求的地址"， resquest_path)
    
    # 遍历路由列表，匹配请求的url
    for path, func in route_list:
        if request_path == path:
            # 找到了指定的路由，执行对于的处理函数
            result = func()
            return result
    else:
        # 没有动态资源数据，返回404状态信息
        result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result
    
    # 判断请求的动态资源路径，选择指定的函数处理对应的动态资源请求
    # if request_path == "/index.html":
        # 获取首页数据
        # result = index()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result
    # else:
        # 没有动态资源数据，返回404状态信息
        # result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result

In [ ]:
# 个人中心数据接口的开发
# framework
# web框架的职责专门负责处理动态资源请求
import time
import pymysql
import josn


# 路由列表，列表里面的每一条记录都是一个路由
route_list = [
#     ("/index.html", index),
#     ("center.html", center)
]


# 定义带有参数的装饰器
def route(path):
    # 装饰器
    def docorator(func):
        # 当执行装饰器的时候就需要把路由添加到路由列表里面
        # 当装饰函数的时候只添加一次路由即可
        route_list.append((path, func))
        
        def inner():
            result = func()
            return result
        
    # 返回一个装饰器
    return decorator


# 获取首页数据
@route("/index.html")
def index():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/index.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    # 创建连接对象
    conn = pymysql.connect(host="localhost", 
                           port="3306", 
                           user="root", 
                           password="mysql", 
                           database="stock_db", 
                           charset="utf8")
    # 获取游标
    cursor = conn.cursor()
    # 准备sql
    sql = "select * from info;"
    # 执行sql
    cursor.ececute(sql)
    # 获取查询结果
    result = cursor.fetchall()
    print(result)
    # 关闭游标
    cursor.close()
    # 关闭连接
    conn.close()
    
    # 遍历每一条数据，完成数据的tr标签的封装
    data = ""
    for row in result:
        data += """<tr>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td><input type="button" value="添加" id="toAdd" name="toAdd" systemidvaule="000007"></td>
                </tr>""" % row
        
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 个人中心数据接口
@route("/center_data.html")
def center_data():
    # 从数据库把数据查询出来，然后把查询处理的数据转成json数据
    # 创建连接对象
    conn = pymysql.connect(host="localhost",
                           port=3306,
                           user="root",
                           password="mysql",
                           database="stock_db",
                           charset="utf8")
    # 获取游标
    cursor = conn.cursor()
    # 准备sql
    sql = '''select i.code, i.short, i.chg, i.turnover, i.price, i.highs, f.note_info
             from info i inner join focus f
             on i.id = f.info_id;
          '''
    # 执行sql
    sursor.execute(sql)
    # 获取查询结果
    result = cursor.fetchall()
    print(result)
    # 把元组转成列表字典
    center_data_list = [{
                        "code":row[0],
                        "short":row[1],
                        "chg":row[2],
                        "turnover":row[3],
                        "price":row[4],
                        "highs":row[5],
                        "note_info":row[6],
                        } for row in result]
    print(center_data_list)
    # 把列表转成json字符串数据
    # ensure_ascii=False 表示在控制台能够显示中文
    json_str = json.dumps(center_data_list, ensure_ascii=False)
    print(json_str)
    print(type(json_str))
    # 关闭游标
    cursor.close()
    # 关闭连接
    conn.close()
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [
        ("Server", "WPS/1.1"),
        # 指定编码格式，因为没有模板文件，可以通过响应头指定编码格式
        ("Content-type", "text/html;charset=utf-8")
    ]
    return status, response_header, json_str


# 获取个人中心数据
@route("/center.html")
def center():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/center.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 处理没有找到的动态资源
def not_found():
    # 状态信息
    status = "404 Not Found"
    # 响应头信息
    response_header = [("Server", "WPS/1.1")]
    # web框架处理后的数据
    data = "Not Found"
    
    # 这里返回的是元组
    return status, response_header, data


# 处理动态资源请求
def handle_request(env):
    # 获取动态的请求资源路径
    request_path = env["request_path"]
    print("动态资源请求的地址"， resquest_path)
    
    # 遍历路由列表，匹配请求的url
    for path, func in route_list:
        if request_path == path:
            # 找到了指定的路由，执行对于的处理函数
            result = func()
            return result
    else:
        # 没有动态资源数据，返回404状态信息
        result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result
    
    # 判断请求的动态资源路径，选择指定的函数处理对应的动态资源请求
    # if request_path == "/index.html":
        # 获取首页数据
        # result = index()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result
    # else:
        # 没有动态资源数据，返回404状态信息
        # result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result

# logging日志
- 记录程序日志信息的目的是:
    - 1. 可以很方便的了解程序的运行情况
    - 2. 可以分析用户的操作行为、喜好等信息
    - 3. 方便开发人员检查bug
- logging日志级别介绍
    - 日志等级可以分为5个，从低到高分别是:
        - 1. DEBUG
        - 2. INFO
        - 3. WARNING
        - 4. ERROR
        - 5. CRITICAL
    - 日志等级说明:
        - DEBUG：程序调试bug时使用
        - INFO：程序正常运行时使用
        - WARNING：程序未按预期运行时使用，但并不是错误，如:用户登录密码错误
        - ERROR：程序出错误时使用，如:IO操作失败
        - CRITICAL：特别严重的问题，导致程序不能再继续运行时使用，如:磁盘空间为空，一般很少使用
        - 默认的是WARNING等级，当在WARNING或WARNING之上等级的才记录日志信息。
        - 日志等级从低到高的顺序是: DEBUG < INFO < WARNING < ERROR < CRITICAL

In [4]:
# logging日志
# logging就是用来记录程序运行时的日志信息
import logging


# 设置logging日志的配置信息
# level 表示设置级别
# %(asctime)s 表示当前时间
# %(filename)s 表示程序文件名
# %(lineno)d 表示符号
# %(levelname)s 表示日志级别
# %(message)s 表示日志信息
logging.basicConfig(level=logging.DEBUG,
                    format="%(asctime)s-%(filename)s[lineno:%(lineno)d]-%(levelname)s-%(message)s",
                    filename="log.txt",
                    filemode="a")
                    
                    
logging.debug("我是一个debug级别的日志信息")
logging.info("我是一个info级别的日志信息")
logging.warning("我是一个warning级别的日志信息")
logging.error("我是一个error级别的日志信息")
logging.critical("我是一个critical级别的日志信息")
                    
# 默认是warning，只有大于等于warning级别的日志才会输出显示

In [ ]:
# web
import socket
import os
import threading
import sys
import framwork
import logging


# 在程序入口模块，设置logging日志的配置信息，只配置一次，整个程序都可以使用，好比单例
logging.basicConfig(level=logging.DEBUG, 
                    format="%(asctime)s-%(filename)s[lineno:%(lineno)d]-%(levelname)s-%(message)s", 
                    filename="log.txt", 
                    filemode="a")


# http协议的Web服务器类
class HttpWebServer(object):
    def __init__(self, port):
    # 创建tcp服务端套接字
    tcp_server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 设置端口号复用，程序退出端口号立即释放
    tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)
    # 绑定端口号
    tcp_server_socket.bind("", port)
    # 设置监听
    tcp_server_socket.listen(128)
    # 把tcp服务器的套接字作为Web服务器对象的属性
    self.tcp_server_socket = tcp_server_socket
    
    # 处理客户端请求
    @staticmethod
    def handle_client_request(new_socket):
        # 接收客户端的请求信息
        recv_data = new_socket.recv(4096)
        # 判断接收的数据长度是否为0
        if len(recv_data) == 0:
            new_socket.close()
            return

        # 对二进制数据进行解码
        recv_centent = recv_data.decode("utf-8")
        print(recv_centent)

        # 对数据按照空格进行分割
        request_list = recv_centent.split(" ", maxsplit=2)
        # 获取请求的资源路径
        request_path = request_list[1]
        print(request_path)

        # 判断请求的是否是根目录，如果是根目录设置返回的信息
        if request_path == "/":
            request_path = "/index.html"
            
        # 判断是否是动态资源请求，以后后缀是.html的请求任务认为是动态资源请求
        if resquest_path.endwith(".html"):
            """动态资源请求"""
            logging.info("动态资源请求地址:" + request_path)
            # 动态资源请求找web框架进行处理，需要把请求参数给web框架
            # 准备给web框架的参数信息，都要放到字典里面
            env = {
                "request_path":resquest_path,
                # 传入请求头信息，额外的参数可以在字典里面再进行添加
            }
            # 使用框架处理动态资源请求
            # 1. web框架需要把处理结果返回给web服务器
            # 2. web服务器负责把返回的结果封装成响应报文发送给浏览器
            status, header, response_body = framwork.handle_request(env)
            print(status, header, respongse_body)
            # 响应行
            response_line = f"HTTP1.1 {status}\r\n"
            # 响应头
            response_header = ""
            for header in headers:
                response_header += f"{header}\r\n"
                
            # 响应报文
            response_data = (response_line +
                             response_header +
                             "\r\n" +
                             response_body).encode("utf8")
            
            # 发送响应报文数据给浏览器
            new_socket.send(response_data)
            # 关闭连接
            new_socket.close()
            
        else:
            """静态资源请求"""
            logging.info("静态资源请求地址:" + request_path)
            # 判断输入的页面是否存在
            # 1. os.path.exits 判断某个文件是否存在
            # os.path.exists("static/" + request_path)
            # 2. try-except

            try:
                # 打开文件读取文件中的数据，提示：这里使用rb模式，兼容打开图片文件
                with open("static" + request_path, "rb") as file: # 这里的file表示打开文件的对象
                    file_data = file.read()
                # 提示：with open 关闭文件这步操作不用程序员来完成，系统帮我们来完成
            except Exception as e:
                # 代码执行到此，说明没有请求的该文件，返回404状态
                # 响应行
                response_line = "HTTP/1.1 200 ok\r\n"
                # 响应头
                response_header = "Server: PWS/1.0\r\n"
                # 读取404页面数据
                with open("static/error.html", "rb") as file:
                    file_data = file.read()
                # 响应体
                response_body = file_data

                # 把数据封装成http响应报文格式的数据
                response = (response_line + 
                            response_header + 
                            "\r\n").encode("utf-8") + response_body
                # 发送给浏览器的相应报文数据
                new_socket.send(response)
            else:    
                # 代码执行到此，说明文件存在，返回200状态信息
                # 响应行
                response_line = "HTTP/1.1 200 ok\r\n"
                # 响应头
                response_header = "Server: PWS/1.0\r\n"
                # 响应体
                response_body = file_data

                # 把数据封装成http响应报文格式的数据
                response = (response_line + 
                            response_header + 
                            "\r\n").encode("utf-8") + response_body

                # 发送给浏览器的响应报文数据
                new_socket.send(response)
            finally:
                # 关闭服务于客户端的套接字
                new_socket.close()
    
    # 启动服务器方法
    def start(self):
         # 循环等待接收客户端的连接请求
        while True:
            # 等待接收客户端的连接请求
            new_socket, ip_port = self.tcp_server_socket.accept()
            # 代码执行到此，说明连接建立成功
            sub_thread = threading.Thread(target=self.handle_client_request, args=(new_socket,))
            # 设置成为守护主线程
            sub_thread.setDaemon(True)
            # 启动子线程执行对应的任务
            sub_thread.start()


def main():
    # 获取终端命令行参数
    # params = sys.argv
    # if len(params) != 2:
        # print("执行的命令格式如下：python3 xxx.py 9000")
        # return
    
    # 判断第二个参数是否都是由数字组成的字符串
    # if not params[1].isdigit():
        # print("执行的命令格式如下：python3 xxx.py 9000")
        # return
    
    # 代码执行到期，说明命令行参数的个数一定是2个，并且第二个参数是由数字组成的字符串
    # port = int(params[1])
    
    # 创建Web服务器
    web_server = HttpWebServer(8000)
    # 启动服务器
    web_server.start()
   
        
# 判断是否是主模块的代码
if __name__ == "__main__":
    main()

In [ ]:
# framework
# web框架的职责专门负责处理动态资源请求
import time
import pymysql
import josn
import logging


# 路由列表，列表里面的每一条记录都是一个路由
route_list = [
#     ("/index.html", index),
#     ("center.html", center)
]


# 定义带有参数的装饰器
def route(path):
    # 装饰器
    def docorator(func):
        # 当执行装饰器的时候就需要把路由添加到路由列表里面
        # 当装饰函数的时候只添加一次路由即可
        route_list.append((path, func))
        
        def inner():
            result = func()
            return result
        
    # 返回一个装饰器
    return decorator


# 获取首页数据
@route("/index.html")
def index():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/index.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    # 创建连接对象
    conn = pymysql.connect(host="localhost", 
                           port="3306", 
                           user="root", 
                           password="mysql", 
                           database="stock_db", 
                           charset="utf8")
    # 获取游标
    cursor = conn.cursor()
    # 准备sql
    sql = "select * from info;"
    # 执行sql
    cursor.ececute(sql)
    # 获取查询结果
    result = cursor.fetchall()
    print(result)
    # 关闭游标
    cursor.close()
    # 关闭连接
    conn.close()
    
    # 遍历每一条数据，完成数据的tr标签的封装
    data = ""
    for row in result:
        data += """<tr>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td>%s</td>
                <td><input type="button" value="添加" id="toAdd" name="toAdd" systemidvaule="000007"></td>
                </tr>""" % row
        
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 个人中心数据接口
@route("/center_data.html")
def center_data():
    # 从数据库把数据查询出来，然后把查询处理的数据转成json数据
    # 创建连接对象
    conn = pymysql.connect(host="localhost",
                           port=3306,
                           user="root",
                           password="mysql",
                           database="stock_db",
                           charset="utf8")
    # 获取游标
    cursor = conn.cursor()
    # 准备sql
    sql = '''select i.code, i.short, i.chg, i.turnover, i.price, i.highs, f.note_info
             from info i inner join focus f
             on i.id = f.info_id;
          '''
    # 执行sql
    sursor.execute(sql)
    # 获取查询结果
    result = cursor.fetchall()
    print(result)
    # 把元组转成列表字典
    center_data_list = [{
                        "code":row[0],
                        "short":row[1],
                        "chg":row[2],
                        "turnover":row[3],
                        "price":row[4],
                        "highs":row[5],
                        "note_info":row[6],
                        } for row in result]
    print(center_data_list)
    # 把列表转成json字符串数据
    # ensure_ascii=False 表示在控制台能够显示中文
    json_str = json.dumps(center_data_list, ensure_ascii=False)
    print(json_str)
    print(type(json_str))
    # 关闭游标
    cursor.close()
    # 关闭连接
    conn.close()
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [
        ("Server", "WPS/1.1"),
        # 指定编码格式，因为没有模板文件，可以通过响应头指定编码格式
        ("Content-type", "text/html;charset=utf-8")
    ]
    return status, response_header, json_str


# 获取个人中心数据
@route("/center.html")
def center():
    # 状态信息
    status = "200 OK"
    # 响应头信息
    response_header = [("Server", "PWS/1.1")]
    # 1. 打开指定模板文件，读取模板文件中的数据
    with open("template/center.html", "r") as file:
        file_data = file.read()
    # 2. 查询数据库，模板里面的模板变量({%content%})替换成以后从数据库里面查询的数据
    
    # web框架处理后的数据
    # 获取当前时间
    data = time.ctime()
    response_body = file_data.replace("{%content%}", data)
    
    # 这里返回的是元组
    return status, response_header, response_body


# 处理没有找到的动态资源
def not_found():
    # 状态信息
    status = "404 Not Found"
    # 响应头信息
    response_header = [("Server", "WPS/1.1")]
    # web框架处理后的数据
    data = "Not Found"
    
    # 这里返回的是元组
    return status, response_header, data


# 处理动态资源请求
def handle_request(env):
    # 获取动态的请求资源路径
    request_path = env["request_path"]
    print("动态资源请求的地址"， resquest_path)
    
    # 遍历路由列表，匹配请求的url
    for path, func in route_list:
        if request_path == path:
            # 找到了指定的路由，执行对于的处理函数
            result = func()
            return result
    else:
        # 没有动态资源数据，返回404状态信息
        result = not_found()
        logging.error("没有设置相关的路由信息:" + request_path)
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        return result
    
    # 判断请求的动态资源路径，选择指定的函数处理对应的动态资源请求
    # if request_path == "/index.html":
        # 获取首页数据
        # result = index()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result
    # else:
        # 没有动态资源数据，返回404状态信息
        # result = not_found()
        # 把处理后的结果返回给web服务器使用，让web服务器拼接响应报文时使用
        # return result